In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
df = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')

In [ ]:
X = df.iloc[:,1:].values
Y = df.iloc[:,0].values

In [ ]:
X = X.reshape(27455,28,28,1)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X[56].reshape(28,28))
plt.show()

In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense,Activation,Flatten,Dropout,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size=0.2)

In [ ]:
xtrain_n = xtrain/xtrain.max()
xtest_n = xtest/xtest.max()

In [ ]:
num_class = 26
ytrain_n = np_utils.to_categorical(ytrain,num_class)
ytest_n = np_utils.to_categorical(ytest,num_class)

In [ ]:
import matplotlib.pyplot as plt
f,ax = plt.subplots(3,5)
f.set_size_inches(15,15)
k = 0
for i in range(3):
    for j in range(5):
        ax[i,j].imshow(xtrain_n[k].reshape(28,28))
        k += 1
    plt.tight_layout()

In [ ]:
wd = 1e-4
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',kernel_regularizer=regularizers.l2(wd) ,input_shape=xtrain_n.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3) , padding='same',kernel_regularizer=regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding='same',kernel_regularizer=regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3) , padding='same',kernel_regularizer=regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3),padding='same',kernel_regularizer=regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3) , padding='same',kernel_regularizer=regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,(3,3),padding='same',kernel_regularizer=regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3) , padding='same',kernel_regularizer=regularizers.l2(wd)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(num_class,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
datagen = ImageDataGenerator(zoom_range=0.2, shear_range=0.3,
                             horizontal_flip = True,
                             width_shift_range=0.1,
                             height_shift_range = 0.1,
                             rotation_range=50,
                             brightness_range=(0.5,1.5))
datagen.fit(xtrain_n)

In [ ]:
learning_rate = 0.01
sgd = keras.optimizers.SGD(lr = learning_rate)

In [ ]:
#Compile your model
model.compile(loss = 'categorical_crossentropy',
              optimizer = sgd,
              metrics = ['accuracy'])

In [ ]:
model.fit_generator(datagen.flow(xtrain_n,ytrain_n), epochs=50, validation_data = (xtest_n,ytest_n))